<a href="https://colab.research.google.com/github/rahulsharma11/S4/blob/main/eva4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [22]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(1, 8, 3, padding=1) #28x28
    self.norm1 = nn.BatchNorm2d(8)
    self.pool1 = nn.MaxPool2d(2, 2)
    self.drop1 = nn.Dropout(0.25)
    self.conv2 = nn.Conv2d(8, 32, 3, padding=1) #14x14
    self.norm2 = nn.BatchNorm2d(32)
    self.pool2 = nn.MaxPool2d(2, 2)
    self.drop2 = nn.Dropout(0.25)
    self.conv3 = nn.Conv2d(32, 32, 3) #7x7
    self.conv4 = nn.Conv2d(32, 16, 3) #5x5
    self.conv5 = nn.Conv2d(16, 10, 3) #3x3
    #self.conv6 = nn.Conv2d(16, 10, 3) #1x1
  
  def forward(self, x):
        x = self.drop1(self.pool1(self.norm1(F.relu(self.conv1(x)))))
        x = self.drop2(self.pool2(self.norm2(F.relu(self.conv2(x)))))
        #x = self.pool2(self.norm2(F.relu(self.conv2(x))))
        x = self.conv5(F.relu(self.conv4(F.relu(self.conv3(x)))))
        #x = self.conv6(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [23]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
       BatchNorm2d-2            [-1, 8, 28, 28]              16
         MaxPool2d-3            [-1, 8, 14, 14]               0
           Dropout-4            [-1, 8, 14, 14]               0
            Conv2d-5           [-1, 32, 14, 14]           2,336
       BatchNorm2d-6           [-1, 32, 14, 14]              64
         MaxPool2d-7             [-1, 32, 7, 7]               0
           Dropout-8             [-1, 32, 7, 7]               0
            Conv2d-9             [-1, 32, 5, 5]           9,248
           Conv2d-10             [-1, 16, 3, 3]           4,624
           Conv2d-11             [-1, 10, 1, 1]           1,450
Total params: 17,818
Trainable params: 17,818
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/ba

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [4]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

In [5]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [24]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(0, 20):
  print (f"epoch : {epoch}")
  train(model, device, train_loader, optimizer, epoch)
  test(model, device, test_loader)

epoch : 0


loss=0.11309096962213516 batch_id=468: 100%|██████████| 469/469 [00:26<00:00, 17.66it/s]



Test set: Average loss: 0.0638, Accuracy: 9807/10000 (98%)

epoch : 1


loss=0.07139860838651657 batch_id=468: 100%|██████████| 469/469 [00:22<00:00, 20.93it/s]



Test set: Average loss: 0.0493, Accuracy: 9856/10000 (99%)

epoch : 2


...


Test set: Average loss: 0.0209, Accuracy: 9930/10000 (99%)

epoch : 18


loss=0.04757966473698616 batch_id=468: 100%|██████████| 469/469 [00:22<00:00, 20.66it/s]



Test set: Average loss: 0.0181, Accuracy: 9942/10000 (99%)

epoch : 19


loss=0.015068882144987583 batch_id=468: 100%|██████████| 469/469 [00:22<00:00, 20.72it/s]



Test set: Average loss: 0.0226, Accuracy: 9930/10000 (99%)

